In [6]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [1]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [7]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/geotweets/geotweets_ny.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "xuqWme99ZBZRzQjJXZqBJexeU"
    consumer_secret = "l2TjeHdl58EdOltAb7fwd1dyZ3D66qu6od3qEG9GwwtA4Y3JjU"
    access_token = "1517264257405968384-pURD4Ci8wM3ExDrTktYBEznIB1axwI"
    access_token_secret = "ExXQDWqQOYyWLG5tBrL4YO9SDAvaDarq2BpZYbFuYORRh"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-75.13911546705614, 40.2462057166038, -72.2743202870626, 41.45331545465915] #NY

    stream_listener = StreamListener(time_limit=600, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS, languages=['en'], encoding="utf-8")

{'id': 1517613860655665158, 'username': 'baroness_beer', 'created_at': 'Fri Apr 22 21:18:34 +0000 2022', 'lng': -74.485366, 'lat': 40.892118499999995, 'text': 'When my daughter was young, the nursing nook was the stands in shea stadium cuz there were no other fans up there @craigcartonlive'}
{'id': 1517613861062463491, 'username': 'quotesfromallah', 'created_at': 'Fri Apr 22 21:18:35 +0000 2022', 'lng': -74.00589108, 'lat': 40.71270142, 'text': 'Just posted a photo @ New York City, N.Y. https://t.co/Vior0HgM49'}
{'id': 1517613867236528130, 'username': 'DFlex2123', 'created_at': 'Fri Apr 22 21:18:36 +0000 2022', 'lng': -74.15394699999999, 'lat': 40.572376000000006, 'text': 'The New York Rangers continue to make history. Artemi Panarin now has the most assists by a NYR forward in a season… https://t.co/5TMRksn2Kj'}
{'id': 1517613874127818753, 'username': 'mikeynyc', 'created_at': 'Fri Apr 22 21:18:38 +0000 2022', 'lng': -74.15394699999999, 'lat': 40.572376000000006, 'text': 'I learned so

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
